In [ ]:
import pandas as pd
import gzip
import os
import numpy as np

 # A la empresa le interesa el rastreo de lo que marketing considera los productos y lo que ventas considera los mejores clientes.

## Mejores Productos
- product_id = {20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032} (diez productos)

## Mejores Clientes
- customer_id = {10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013} (doce clientes)

Total de <producto, cliente> a predecir = 10 * 12 = 120

# Objetivo
- Es el 01-enero-2020 a las 00:01 y disponibilizamos las ventas del periodo 2021912.
- El 02-enero a las 18:00 nos deben entregar:
  - El primer forecast de ventas para cada producto que se harán durante el mes 202002, de forma que nuestras plantas puedan fabricarlos durante el mes de 202001.
  - El segundo forecast es las ventas esperadas en 202002, para los 120 pares de <mejores_clientes, mejores_productos>.

In [ ]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")
os.chdir("C:/diego_tools/labo3/dataset")
####################################################

In [ ]:
arch_sellout = "tb_sellout_02.txt.gz"
arch_maestro_prod = "maestro_productos_depurado.csv"
arch_exogenas = "emp3_exogenas.csv"
arch_prod_ids_prediccion = "productos_a_predecir.csv"

In [ ]:
# Variables para definir que atributos se descartan
meses_para_control_vigencia = [201904,201905,201906] #meses en los cuales deben aparecer los productos para ser considerados vigentes (NO discontinuados) y ser tomados en la prediccion
tope_fecha_historia = 201902 #los productos que aparezcan desde este mes (inclusive) en adelante, se excluyen por tener poca historia de ventas

In [ ]:
def diferencia_meses(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [ ]:
clientes_estrella = [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013]

## Sellout

In [ ]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [ ]:
df_sellout.info()

In [ ]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

**--> sin duplicados**

In [ ]:
df_sellout.isna().sum()

**--> sin nulos**

In [ ]:
df_sellout.periodo.unique()                

In [ ]:
len(df_sellout.product_id.unique())          

--> algunos productos no van a tener descripción

In [ ]:
len(df_sellout.customer_id.unique())            

In [ ]:
df_sellout.plan_precios_cuidados.unique()          

In [ ]:
df_sellout[df_sellout.tn==0]

--> si no hay ventas, no hay registro en 0 (directamente no hay registro)

In [ ]:
df_sellout.head()

In [ ]:
# Como control, sumo tns
tn_suma_original = round(sum(df_sellout.tn))
print("Toneladas Total Control:", round(sum(df_sellout.tn),0))

## Descarte de Productos que no hay que predecir

In [ ]:
# Discontinuados (sin ventas en 3 meses mas adelante)
product_ids_vigentes =  df_sellout[df_sellout.periodo.isin(meses_para_control_vigencia)].product_id.unique()
print("Vigentes:", len(product_ids_vigentes))

product_ids_discontinuados = set(df_sellout.product_id.unique()).difference(set(product_ids_vigentes))
print("Discontinuados:", len(product_ids_discontinuados))

#Sin historia suficiente (minima fecha >= 201902)
product_ids_sin_hist_sufic = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic[product_ids_sin_hist_sufic.periodo>=tope_fecha_historia]
print(product_ids_sin_hist_sufic.head(5))
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic.product_id.unique()
print("Menos 3 meses hist:", len(product_ids_sin_hist_sufic))

#Interseccion y union
print("Interseccion: ", len((set(product_ids_discontinuados).intersection(set(product_ids_sin_hist_sufic)))))
print("Union: ", len((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic)))))


product_ids_para_predecir = set(df_sellout.product_id.unique()).difference((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic))))
print("\nTotal:", len(product_ids_para_predecir))

In [ ]:
#Se guardan en un csv aparte
df_prods_prediccion = pd.DataFrame(data={"product_id":list(product_ids_para_predecir)})
df_prods_prediccion.to_csv(arch_prod_ids_prediccion, index=False)

## Completado de Períodos sin Ventas

* Los pares <cliente,producto> que no aparecen un mes van a ser completados con un registro en 0
* Sin embargo, se completará a partir de su primer mes (y no para atrás)
* Esto aplicará para el primer mes del cliente y del producto

In [ ]:
periodos = df_sellout.periodo.unique()
cant_periodos = len(periodos)
periodos, cant_periodos

In [ ]:
productos = df_sellout.product_id.unique()
cant_productos = len(productos)
cant_productos

In [ ]:
clientes = df_sellout.customer_id.unique()
cant_clientes = len(clientes)
cant_clientes

In [ ]:
len(df_sellout),cant_productos*cant_periodos*cant_clientes

**--> no todos los productos y clientes están en todos los períodos (2.95M vs 26.5M)**

In [ ]:
# Obtengo el primer mes de cada producto
df_primer_mes_prod = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_prod = df_primer_mes_prod.rename(columns={"periodo":"primer_periodo_prod"})
df_primer_mes_prod.tail()

In [ ]:
# Obtengo el primer mes de cada cliente
df_primer_mes_cliente = df_sellout.groupby("customer_id").agg({"periodo":"min"}).reset_index()
df_primer_mes_cliente = df_primer_mes_cliente.rename(columns={"periodo":"primer_periodo_cliente"})
df_primer_mes_cliente.tail()

In [ ]:
# Se va a poner 0 a todos los periodos donde el producto no se vendio
df_cartesiano = pd.DataFrame(data={"product_id":productos}).merge(pd.DataFrame(data={"periodo":periodos}), how='cross')
df_cartesiano = df_cartesiano.merge(pd.DataFrame(data={"customer_id":clientes}), how='cross')

df_cartesiano["cero_ventas"] = 0
len(df_cartesiano)

In [ ]:
df_cartesiano.info()

In [ ]:
df_cartesiano.head()

In [ ]:
df_sellout_complet = df_cartesiano.merge(df_sellout, how="left",on=["product_id","customer_id","periodo"])
len(df_sellout_complet)

In [ ]:
df_sellout_complet.head()

In [ ]:
df_sellout_complet.isna().sum()

In [ ]:
# Imputo
df_sellout_complet.cero_ventas = np.where(df_sellout_complet.tn.isna(),1,0)
df_sellout_complet.tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.tn)
df_sellout_complet.cust_request_tn = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_tn)
df_sellout_complet.cust_request_qty = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.cust_request_qty)
df_sellout_complet.plan_precios_cuidados = np.where(df_sellout_complet.cero_ventas==1,0,df_sellout_complet.plan_precios_cuidados)

In [ ]:
df_sellout_complet.isna().sum()

In [ ]:
df_sellout_complet.cero_ventas.sum(),len(df_cartesiano)-len(df_sellout)

In [ ]:
# Ahora, cruzo con el primer mes (para luego borrar aquellos registros previos al primer mes de datos)
print(len(df_sellout_complet))
df_sellout_complet_desde_1er_mes = df_sellout_complet.merge(df_primer_mes_prod,on="product_id",how="inner")
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.merge(df_primer_mes_cliente,on="customer_id",how="inner")
print(len(df_sellout_complet_desde_1er_mes))

In [ ]:
# Borro los previos al primer mes
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_prod) & (df_sellout_complet_desde_1er_mes.periodo >= df_sellout_complet_desde_1er_mes.primer_periodo_cliente)]

print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.tail(10)

--> **Quedan 19.6M de registros**

### --> La que sigue es la parte que más demora...

In [ ]:
df_sellout_complet_desde_1er_mes['periodo_fecha'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['periodo'], format='%Y%m')

# Se agrega una variable que tenga el mes y tambien la cantidad de meses de historia del producto
df_sellout_complet_desde_1er_mes["mes"] = pd.DatetimeIndex(df_sellout_complet_desde_1er_mes.periodo_fecha).month
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_prod'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_prod'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_prod"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_prod"]),axis=1)
df_sellout_complet_desde_1er_mes['primer_periodo_fecha_cliente'] = pd.to_datetime(df_sellout_complet_desde_1er_mes['primer_periodo_cliente'], format='%Y%m')
df_sellout_complet_desde_1er_mes["meses_historia_cliente"]=df_sellout_complet_desde_1er_mes.apply(lambda row: diferencia_meses(row["periodo_fecha"],row["primer_periodo_fecha_cliente"]),axis=1)

#Se agrega columna de "cliente_estrella" y par producto-cliente
df_sellout_complet_desde_1er_mes["cliente_estrella"]=np.where(df_sellout_complet_desde_1er_mes.customer_id.isin(clientes_estrella),1,0)
df_sellout_complet_desde_1er_mes["prod_cust"] = df_sellout_complet_desde_1er_mes.product_id.astype(str) + "-" + df_sellout_complet_desde_1er_mes.customer_id.astype(str)
# Dejo unicamente las columnas de meses_historia
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.drop(columns=["primer_periodo_prod","primer_periodo_cliente","primer_periodo_fecha_prod","primer_periodo_fecha_cliente"])

In [ ]:
df_sellout_complet_desde_1er_mes.head(10)

### Comprobación de casos puntuales

In [ ]:
df_sellout[(df_sellout.customer_id==10637)&(df_sellout.product_id==20741)]

In [ ]:
df_sellout[(df_sellout.customer_id==10637)].periodo.min()

In [ ]:
df_sellout[(df_sellout.product_id==20741)].periodo.min()

In [ ]:
df_sellout_complet_desde_1er_mes[(df_sellout_complet_desde_1er_mes.customer_id==10637)&(df_sellout_complet_desde_1er_mes.product_id==20741)]

**--> se generó OK, desde 201709 en adelante únicamente**

## Incorporo Maestro y Exógenas

In [ ]:
df_maestro_prod = pd.read_csv(arch_maestro_prod)

In [ ]:
df_maestro_prod.info()

In [ ]:
df_maestro_prod.head()

In [ ]:
prods_desconocidos = set(df_sellout_complet_desde_1er_mes.product_id).difference(set(df_maestro_prod.product_id))
print(len(prods_desconocidos))

In [ ]:
df_tn_prod_desc = df_sellout_complet_desde_1er_mes[df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos)]
df_tn_prod_desc.tn.sum()

In [ ]:
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes, df_maestro_prod, on='product_id', how='left')
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes.head()

In [ ]:
df_sellout_complet_desde_1er_mes.cat1 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat1)
df_sellout_complet_desde_1er_mes.cat2 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat2)
df_sellout_complet_desde_1er_mes.cat3 = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.cat3)
df_sellout_complet_desde_1er_mes.brand = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),"desconocida",df_sellout_complet_desde_1er_mes.brand)
df_sellout_complet_desde_1er_mes.sku_size = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.sku_size)
df_sellout_complet_desde_1er_mes.producto_estrella = np.where(df_sellout_complet_desde_1er_mes.product_id.isin(prods_desconocidos),0,df_sellout_complet_desde_1er_mes.producto_estrella)

In [ ]:
df_exogenas = pd.read_csv(arch_exogenas)
df_exogenas.periodo_fecha = pd.to_datetime(df_exogenas.periodo_fecha)

df_exogenas.info()

In [ ]:
# Le agregamos las exogenas
print(len(df_sellout_complet_desde_1er_mes))
df_sellout_complet_desde_1er_mes = pd.merge(df_sellout_complet_desde_1er_mes,df_exogenas,on="periodo_fecha",how="left")
print(len(df_sellout_complet_desde_1er_mes))

In [ ]:
# Como control, sumo tns
print("Toneladas Total Control:", round(sum(df_sellout_complet_desde_1er_mes.tn),0),tn_suma_original==round(sum(df_sellout_complet_desde_1er_mes.tn),0))

In [ ]:
df_sellout_complet_desde_1er_mes.info()

In [ ]:
# Ordeno para que luego el FE funcione correctamente usando SHIFT
df_sellout_complet_desde_1er_mes = df_sellout_complet_desde_1er_mes.sort_values(by=["product_id","customer_id","periodo"],ascending=True)

In [ ]:
df_sellout_complet_desde_1er_mes.head(50)

In [ ]:
df_sellout_complet_desde_1er_mes.isna().sum()

In [ ]:
# Exportar el DataFrame a un archivo CSV
df_sellout_complet_desde_1er_mes.to_csv("emp3_sellout_base.csv", index=False)